<a href="https://colab.research.google.com/github/mohitsharmas97/pytorch_notebooks/blob/main/LSTM_day12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk #natural language toolkit

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset,DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [ ]:
document="""About Data Science Mentorship Program
What is the course fee for  Data Science Mentorship Program (DSMP 2.0)
The total course fee for the DSMP course is Rs 10,600. This includes the fee for both DSMP 1.0 and DSMP 2.0
What is the total duration of the course?
The total duration of the course is about 6-8 months.
Are Deep Learning and NLP a part of the DSMP 2.0  course?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes, all our sessions are recorded, so even if you miss a session you can login into our portal and watch the recording as per your convenience.
Where can I find the class schedule?
You will find the class schedule in your course dashboard once you enroll for the course.
"""

In [ ]:
#tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
#tokenize
tokens=word_tokenize(document.lower())

In [ ]:
#build vocab
vocab={'<unk>':0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token]=len(vocab)

vocab


{'<unk>': 0,
 'about': 1,
 'data': 2,
 'science': 3,
 'mentorship': 4,
 'program': 5,
 'what': 6,
 'is': 7,
 'the': 8,
 'course': 9,
 'fee': 10,
 'for': 11,
 '(': 12,
 'dsmp': 13,
 '2.0': 14,
 ')': 15,
 'total': 16,
 'rs': 17,
 '10,600.': 18,
 'this': 19,
 'includes': 20,
 'both': 21,
 '1.0': 22,
 'and': 23,
 'duration': 24,
 'of': 25,
 '?': 26,
 '6-8': 27,
 'months': 28,
 '.': 29,
 'are': 30,
 'deep': 31,
 'learning': 32,
 'nlp': 33,
 'a': 34,
 'part': 35,
 'no': 36,
 ',': 37,
 'not': 38,
 '’': 39,
 's': 40,
 'curriculum': 41,
 'if': 42,
 'i': 43,
 'miss': 44,
 'live': 45,
 'session': 46,
 'will': 47,
 'get': 48,
 'recording': 49,
 'yes': 50,
 'all': 51,
 'our': 52,
 'sessions': 53,
 'recorded': 54,
 'so': 55,
 'even': 56,
 'you': 57,
 'can': 58,
 'login': 59,
 'into': 60,
 'portal': 61,
 'watch': 62,
 'as': 63,
 'per': 64,
 'your': 65,
 'convenience': 66,
 'where': 67,
 'find': 68,
 'class': 69,
 'schedule': 70,
 'in': 71,
 'dashboard': 72,
 'once': 73,
 'enroll': 74}

In [ ]:
len(vocab)

75

In [ ]:
#extract sentences from data
input_sentences=[]

for sentence in document.split("\n"):
  input_sentences

In [ ]:
input_sentences=document.split("\n")

In [ ]:
def text_to_indices(sentence,vocab):
  numerical_sentence=[]
  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])
  return numerical_sentence


In [ ]:
input_numerical_sentences=[]
for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()),vocab))



In [ ]:
len(input_numerical_sentences)

12

In [ ]:
training_sequence=[]
for sentence in input_numerical_sentences:

  for i in range(1,len(sentence)):
    training_sequence.append(sentence[:i+1])




In [ ]:
len(training_sequence)

153

In [ ]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [1, 2, 3, 4], [1, 2, 3, 4, 5], [6, 7]]

In [ ]:
len_list=[]
for sequence in training_sequence:
  len_list.append(len(sequence))
max(len_list)

30

In [ ]:
training_sequence[0]

[1, 2]

In [ ]:
padded_training_sequence=[]
for sequence in training_sequence:
  padded_training_sequence.append([0]*(max(len_list)-len(sequence))+sequence)

In [ ]:
len(padded_training_sequence[2])

30

In [ ]:
padded_training_sequence=torch.tensor(padded_training_sequence,dtype=torch.long)

In [ ]:
padded_training_sequence.shape

torch.Size([153, 30])

In [ ]:
X=padded_training_sequence[:,:-1]
y=padded_training_sequence[:,-1]


In [ ]:
class customDataset(Dataset):
  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [ ]:
dataset=customDataset(X,y)

In [ ]:
len(dataset)

153

In [ ]:
dataloader=DataLoader(dataset,batch_size=32,shuffle=True)

In [ ]:
for input,output in dataloader:
  print(input,output)

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0, 50, 37, 51],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 50,
         37, 51, 52, 53, 30, 54, 37, 55, 56, 42, 57],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         50, 37, 51, 52, 53, 30, 54, 37, 55, 56, 42],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  8, 16,  9, 10, 11,  8, 13,  9,
          7, 17, 18, 19, 20,  8, 10, 11, 21, 13, 22],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 36, 37, 33, 23,
         31, 32, 21, 30, 38, 34, 35, 25, 19,  5, 39],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 57, 47, 68,
          8, 69, 70, 71, 65,  9, 72, 73, 57, 74, 11],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, 36, 37,
         33, 23, 31, 32, 21, 30, 38, 34, 35, 25, 19],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0

In [ ]:
X.shape

torch.Size([153, 29])

In [ ]:
y.shape

torch.Size([153])

In [ ]:
dataset[0]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 1]),
 tensor(2))

In [ ]:
class LSTMModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,100)
    self.lstm=nn.LSTM(100,150,batch_first=True)
    self.fc=nn.Linear(150,vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [ ]:
model=LSTMModel(len(vocab))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model.to(device)

LSTMModel(
  (embedding): Embedding(75, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=75, bias=True)
)

In [ ]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# training loop

for epoch in range(epochs):
  total_loss = 0

  for batch_x, batch_y in dataloader:

    batch_x, batch_y = batch_x.to(device), batch_y.to(device)

    optimizer.zero_grad()

    output = model(batch_x)

    loss = criterion(output, batch_y)

    loss.backward()

    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")

Epoch: 1, Loss: 21.5686
Epoch: 2, Loss: 20.8926
Epoch: 3, Loss: 20.2456
Epoch: 4, Loss: 19.4595
Epoch: 5, Loss: 18.4883
Epoch: 6, Loss: 17.4044
Epoch: 7, Loss: 16.3934
Epoch: 8, Loss: 15.2340
Epoch: 9, Loss: 14.1345
Epoch: 10, Loss: 12.9762
Epoch: 11, Loss: 11.8407
Epoch: 12, Loss: 10.7640
Epoch: 13, Loss: 9.7527
Epoch: 14, Loss: 8.7648
Epoch: 15, Loss: 7.8242
Epoch: 16, Loss: 7.0226
Epoch: 17, Loss: 6.2331
Epoch: 18, Loss: 5.5649
Epoch: 19, Loss: 4.9899
Epoch: 20, Loss: 4.4126
Epoch: 21, Loss: 3.9491
Epoch: 22, Loss: 3.5410
Epoch: 23, Loss: 3.1571
Epoch: 24, Loss: 2.8732
Epoch: 25, Loss: 2.6010
Epoch: 26, Loss: 2.3334
Epoch: 27, Loss: 2.1156
Epoch: 28, Loss: 1.9214
Epoch: 29, Loss: 1.7804
Epoch: 30, Loss: 1.6338
Epoch: 31, Loss: 1.5274
Epoch: 32, Loss: 1.4120
Epoch: 33, Loss: 1.3180
Epoch: 34, Loss: 1.2252
Epoch: 35, Loss: 1.1543
Epoch: 36, Loss: 1.0854
Epoch: 37, Loss: 1.0334
Epoch: 38, Loss: 0.9764
Epoch: 39, Loss: 0.9333
Epoch: 40, Loss: 0.8930
Epoch: 41, Loss: 0.8398
Epoch: 42, Lo

In [ ]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (30 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to device
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  padded_text = padded_text.to(device)

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]

In [ ]:
prediction(model, vocab, "About Data Science mentorship program ( dsmp")

'About Data Science mentorship program ( dsmp 2.0'

In [ ]:
import time

num_tokens = 10
input_text = "hi how are"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)


hi how are deep
hi how are deep learning
hi how are deep learning and
hi how are deep learning and nlp
hi how are deep learning and nlp a
hi how are deep learning and nlp a part
hi how are deep learning and nlp a part of
hi how are deep learning and nlp a part of the
hi how are deep learning and nlp a part of the dsmp
hi how are deep learning and nlp a part of the dsmp 2.0


In [ ]:
dataloader1 = DataLoader(dataset, batch_size=32, shuffle=False)

In [ ]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 98.04%
